In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries


In [2]:

!pip install --upgrade transformers
!pip install datasets
!pip install rouge_score
!pip install rouge
!pip install sentencepiece
!pip install Dataset
from datasets import load_dataset, load_metric, Dataset

     |████████████████████████████████| 4.0 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 71.4 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
     |████████████████████████████████| 6.5 MB 74.7 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 7.8 MB/s 
     |████████████████████████████████| 136 kB 89.0 MB/s 
     |████████████████████████████████| 212 kB 84.2 MB/s 
     |████████████████████████████████| 1.1 MB 71.7 MB/s 
     |████████████████████████████████| 127 kB 85.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 70.2 MB/s 
     |████████████████████████████████| 144 kB 88.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

## Dataset Preparation

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/NN_Project/amazon_review_dataset_processed.csv"
df = pd.read_csv(path)
amazon = Dataset.from_pandas(df)
amazon.shape

(11848, 3)

In [4]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric
import torch
import numpy as np
import torch
model_name = 'facebook/bart-large-cnn'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = transformers.BartTokenizer.from_pretrained(model_name)

model = transformers.BartForConditionalGeneration.from_pretrained(model_name).to(torch_device)

max_source_length = 512
max_target_length = 175

def preprocess_function(reviews):
  input_sequences = reviews['reviewText']
  inputs = [sequence for sequence in input_sequences]
  model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, padding = True)
  summaries = reviews['summary']
  labels = tokenizer(summaries, max_length=max_target_length, truncation=True, padding = True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

tokenized_amazon = amazon.map(preprocess_function, batched=True)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [5]:

NotTest_Test = tokenized_amazon.train_test_split(test_size=0.1,seed=42)
NotTest = NotTest_Test["train"]
test = NotTest_Test["test"]

Train_Val = NotTest.train_test_split(test_size=0.1,seed=42)
train = Train_Val["train"]
val = Train_Val["test"]

print(train.shape, val.shape, test.shape)

(9596, 6) (1067, 6) (1185, 6)


## Training

In [6]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train,
    eval_dataset = val,
    tokenizer = tokenizer,
    data_collator = data_collator
)

trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, reviewText, Unnamed: 0. If summary, reviewText, Unnamed: 0 are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9596
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 28788


Epoch,Training Loss,Validation Loss
1,1.764000,1.723398
2,1.255300,1.678363
3,0.999900,1.783351


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=28788, training_loss=1.381941082577946, metrics={'train_runtime': 9248.9583, 'train_samples_per_second': 3.113, 'train_steps_per_second': 3.113, 'total_flos': 3.119330364280013e+16, 'train_loss': 1.381941082577946, 'epoch': 3.0})

In [8]:
trainer.save_model("/content/drive/MyDrive/finetunedModelBartLargeCNN")

Saving model checkpoint to /content/drive/MyDrive/finetunedModelBartLargeCNN
Configuration saved in /content/drive/MyDrive/finetunedModelBartLargeCNN/config.json
Model weights saved in /content/drive/MyDrive/finetunedModelBartLargeCNN/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/finetunedModelBartLargeCNN/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/finetunedModelBartLargeCNN/special_tokens_map.json


In [9]:
finetuned = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/finetunedModelBartLargeCNN")

loading configuration file /content/drive/MyDrive/finetunedModelBartLargeCNN/config.json
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/finetunedModelBartLargeCNN",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    

## Generate Summaries

In [10]:
import math
all_outputs = []
x = torch.Tensor(test['input_ids']).long()

In [11]:
outputs = finetuned.generate(x[:20], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [12]:
outputs = finetuned.generate(x[20:40], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [13]:

outputs = finetuned.generate(x[40:60], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [14]:
outputs = finetuned.generate(x[60:80], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [15]:
outputs = finetuned.generate(x[80:100], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [16]:
outputs = finetuned.generate(x[100:120], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [17]:
outputs = finetuned.generate(x[120:140], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [18]:
outputs = finetuned.generate(x[140:160], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [19]:
outputs = finetuned.generate(x[160:180], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [20]:
outputs = finetuned.generate(x[180:200], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [23]:
df_result = pd.DataFrame({'review':test['reviewText'][:200],'generated_summaries':flattened_outputs,'target_summaries':test['summary'][:200]})
df_result.to_csv('/content/drive/MyDrive/NN_Project/bart_large_cnn.csv') 

## Evaluation

In [21]:
metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

flattened_outputs = np.array(all_outputs).flatten()
calc_rouge_scores(test['summary'][:200], flattened_outputs)




{'rouge1': 23.0, 'rouge2': 10.3, 'rougeL': 20.7, 'rougeLsum': 20.7}

In [22]:
metric = load_metric('rouge')
flattened_outputs = np.array(all_outputs).flatten()
values = metric.compute(predictions=test['summary'][:200], references = flattened_outputs, use_stemmer=True)
values

{'rouge1': AggregateScore(low=Score(precision=0.2224269371065812, recall=0.19178762604301006, fmeasure=0.20265563713735654), mid=Score(precision=0.2509399471679853, recall=0.2191627393933896, fmeasure=0.22985089703565523), high=Score(precision=0.28353164959019744, recall=0.2517504102756764, fmeasure=0.2626524672411639)),
 'rouge2': AggregateScore(low=Score(precision=0.08333715737897719, recall=0.0742277764106808, fmeasure=0.07704126798675866), mid=Score(precision=0.11070768896136537, recall=0.10048515944839471, fmeasure=0.1032976861978824), high=Score(precision=0.14424929948523696, recall=0.12999040892726552, fmeasure=0.13339067061808238)),
 'rougeL': AggregateScore(low=Score(precision=0.19644085167928044, recall=0.1699668468485419, fmeasure=0.17882984994614448), mid=Score(precision=0.22549454037534544, recall=0.19654173733043714, fmeasure=0.20654495864592176), high=Score(precision=0.25631630893884677, recall=0.22468242607066138, fmeasure=0.23562304925565203)),
 'rougeLsum': AggregateS